In [1]:
!pip install yfinance

In [2]:
import warnings
warnings.filterwarnings('ignore')

import yfinance as yf
import pandas as pd
import numpy as np
import altair as alt
import datetime
import project_functions2 as pf

from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
stock_list = ['AMZN', 'AAPL', 'FB','GOOG', 'MSFT', 'TSLA']

In [5]:
stock_list_df = pd.read_csv('/content/drive/MyDrive/SENG474_Project/data/nasdaq_list.csv')
stock_list_df = stock_list_df[stock_list_df['Market Cap'] > 0]
stock_list_df = stock_list_df[stock_list_df['IPO Year'] < 2018]
#stock_list_df = stock_list_df[stock_list_df['Volume'] > 10000]
stock_list_df['Last Sale'] = stock_list_df['Last Sale'].str.slice(start=1)
stock_list_df['Last Sale'] = stock_list_df['Last Sale'].astype(float)
stock_list_df = stock_list_df[stock_list_df['Volume']*stock_list_df['Last Sale']> 10000000]
#stock_list_df = stock_list_df[stock_list_df['Sector'] == 'Technology']
stock_list = list(stock_list_df['Symbol'])
stock_list_df

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
8,AAPL,Apple Inc. Common Stock,121.21,0.620,0.514%,2.101459e+12,United States,1980.0,93940602,Technology,Computer Manufacturing
10,ABCB,Ameris Bancorp Common Stock,54.94,2.260,4.29%,3.824866e+09,United States,1994.0,612835,Finance,Major Banks
24,ACAD,ACADIA Pharmaceuticals Inc. Common Stock,25.86,-0.160,-0.615%,4.138828e+09,United States,1985.0,1388296,Health Care,Major Pharmaceuticals
43,ACLS,Axcelis Technologies Inc. Common Stock,39.19,2.130,5.747%,1.321961e+09,United States,2000.0,371793,Technology,Industrial Machinery/Components
44,ACMR,ACM Research Inc. Class A Common Stock,83.36,-0.640,-0.762%,1.562419e+09,United States,2017.0,230639,Technology,Industrial Machinery/Components
...,...,...,...,...,...,...,...,...,...,...,...
3955,ZG,Zillow Group Inc. Class A Common Stock,132.08,0.730,0.556%,3.178632e+10,United States,2011.0,1137494,Miscellaneous,Business Services
3956,ZGNX,Zogenix Inc. Common Stock,18.76,-0.055,-0.292%,1.045599e+09,United States,2010.0,1071416,Health Care,Major Pharmaceuticals
3969,ZKIN,ZK International Group Co. Ltd Ordinary Share,9.80,-0.550,-5.314%,1.622688e+08,China,2017.0,3000539,Capital Goods,Metal Fabrications
3970,ZLAB,Zai Lab Limited American Depositary Shares,128.42,-1.660,-1.276%,9.679723e+09,China,2017.0,349013,Health Care,Major Pharmaceuticals


In [6]:
stock_objects = {}
for stock in stock_list:
    stock_objects[stock] = yf.Ticker(stock)

In [7]:
def trading_sim_LRM(stock_objects, split_time, time_shift):
    columns = ['Date', 'Fund Value', 'Cash']
    stock_names = []
    proj_buy = 0
    curr_cash = 10000
    curr_shares = {}
    drop_list = [ 'Volume', 'Dividends', 'Stock Splits',
                 '5 Day Open Mean', '5 Day High Mean', '5 Day Low Mean',
                 '5 Day Close Mean', '5 Day Volume Mean', '5 Day Open Var',
                 '5 Day High Var', '5 Day Low Var', '5 Day Close Var',
                 '5 Day Volume Var','10 Day Open Mean', '10 Day High Mean', 
                 '10 Day Low Mean','10 Day Close Mean', '10 Day Volume Mean', 
                 '10 Day High Var', '10 Day Low Var', '10 Day Close Var',
                 '10 Day Volume Var', '10 Day High', '10 Day Low', 
                 '20 Day Open Mean', '20 Day High Mean', '20 Day Low Mean',
                 '20 Day Close Mean', '20 Day Volume Mean', '20 Day Open Var',
                 '20 Day High Var', '20 Day Low Var', '20 Day Close Var',
                 '20 Day Volume Var', '10 Day Open Var']
    
    stock_dfs = {}
    for key in stock_objects:
        stock_dfs[key] = stock_objects[key].history(start='1990-01-01', end='2021-03-31')
        
    #stock_investing = {}
    #for key in stock_objects:
    #    stock_investing[key] = pd.read_csv('/content/drive/MyDrive/SENG474_Project/data/sentiment/investing_'+key+'_sentiment.csv')
    #    stock_investing[key].set_index('date', inplace=True)
    #stock_stocks = {}
    #for key in stock_objects:
    #    stock_stocks[key] = pd.read_csv('/content/drive/MyDrive/SENG474_Project/data/sentiment/stocks_'+key+'_sentiment.csv')
    #    stock_stocks[key].set_index('date', inplace=True)
    
    for key in stock_dfs:
        stock_dfs[key] = pf.rolling_aves(stock_dfs[key])
        stock_dfs[key].drop(drop_list, axis=1, inplace=True)
        #stock_dfs[key] = stock_dfs[key].merge(stock_investing[key], how='left', left_index=True, right_index=True)
        #stock_dfs[key] = stock_dfs[key].merge(stock_stocks[key], how='left', left_index=True, right_index=True)
        stock_dfs[key].replace([np.inf, -np.inf], np.nan, inplace=True)
        stock_dfs[key].fillna(0, inplace=True)
        stock_dfs[key] = pf.future_low_setup(stock_dfs[key], 1)
        stock_dfs[key] = pf.future_close_setup(stock_dfs[key], time_shift)

    combine_df = pf.combiner(stock_dfs)
        
    test_dfs = stock_dfs
    
    for key in test_dfs:
        test_dfs[key] = test_dfs[key].tail(split_time)
        curr_shares[key] = 0
        stock_names.append(key)
    
    columns = columns + stock_names
    cash_df = pd.DataFrame(columns=columns)
    curr_line = [combine_df.index[int(len(combine_df) - (split_time * len(stock_dfs)))], curr_cash, curr_cash] + len(stock_names)*[0]
    cash_df.loc[len(cash_df)] = curr_line
    
    while split_time > time_shift:
        #print('Days left: ' +str(split_time))
        max_stock = ''
        max_stock_gain = 0
        split_mark = int(len(combine_df) - (split_time * len(stock_dfs)))
        
        X_train, y_train, X_test, y_test = pf.multi_stock_train_test_split(combine_df, split_time, stock_dfs)
        low_train = X_train.iloc[:,-1]
        low_test = X_test.iloc[:,-1]
        X_train.drop(['Low in 1 Days'], axis=1, inplace=True)
        X_test.drop(['Low in 1 Days'], axis=1, inplace=True)
        stock_model = LinearRegression().fit(X_train, y_train)
        low_model = LinearRegression().fit(X_train, low_train)
        
        for key in test_dfs:
            X = test_dfs[key].iloc[:,:-2]
            stock_pred = stock_model.predict(X.head(1))
            stock_pred = (float(stock_pred) - X['Close'][0]) / X['Close'][0]
            low_pred = low_model.predict(X.head(1))
            if stock_pred > max_stock_gain and low_pred > 0:
                max_stock = key
                max_stock_gain = stock_pred
                max_stock_low_pred = low_pred
        
        for key in test_dfs:
            test_dfs[key] = test_dfs[key].iloc[1:]
        
        
        if max_stock_gain > 0:
            if test_dfs[max_stock]['Open'][0] < max_stock_low_pred:
                proj_buy = test_dfs[max_stock]['Open'][0]
            else:    
                proj_buy = float((test_dfs[max_stock]['Open'][0] - max_stock_low_pred)*0.5 + test_dfs[max_stock]['Low'][0])
            #print(test_dfs[max_stock].index[0])
            #if max_stock_low_pred >= test_dfs[max_stock]['Open'][0]:
            #   print('Buy at open')
            #print(float(proj_buy - test_dfs[max_stock]['Low'][0]))
            if curr_shares[max_stock] == 0:
                for key in curr_shares:
                    if curr_shares[key] > 0:
                        curr_cash -= 6.95
                    curr_cash += curr_shares[key]*test_dfs[key]['Open'][0]
                    curr_shares[key] = 0

                if proj_buy >= test_dfs[max_stock]['Low'][0]:
                    curr_cash -= 6.95
                    curr_shares[max_stock] = curr_cash // proj_buy
                    curr_cash -= curr_shares[max_stock]*proj_buy
            elif proj_buy >= test_dfs[max_stock]['Low'][0]:
                if curr_cash > 70:
                    curr_cash -= 6.95
                    curr_shares[max_stock] += curr_cash // proj_buy
                    curr_cash -= (curr_cash // proj_buy) * proj_buy

            '''if proj_buy*0.9 > test_dfs[max_stock]['Low'][0]:
                if curr_shares[max_stock] > 0:
                    curr_cash -= 6.95
                curr_cash += curr_shares[max_stock]*proj_buy*0.9
                curr_shares[max_stock] = 0'''

        else:
            for key in curr_shares:
                if curr_shares[key] > 0:
                    curr_cash -= 6.95
                curr_cash += curr_shares[key]*test_dfs[key]['Open'][0]
                curr_shares[key] = 0
        
        curr_line = [X_test.index[len(stock_names)], curr_cash ,curr_cash] + len(stock_names)*[0]
        cash_df.loc[len(cash_df)] = curr_line
        for key in curr_shares:
            cash_df.iloc[-1, cash_df.columns.get_loc(key)] = curr_shares[key]
            cash_df.iloc[-1, cash_df.columns.get_loc('Fund Value')] += curr_shares[key]*test_dfs[key]['Open'][0]
        for key in test_dfs:
            test_dfs[key] = test_dfs[key].iloc[time_shift-1:]
        
        split_time -= time_shift

    return cash_df

In [8]:
fund_df = trading_sim_LRM(stock_objects, 292, 1)
fund_df.set_index('Date', inplace=True) # on big 6
fund_df #57904.5 --> 1 day

,Fund Value,Cash,AAPL,ABCB,ACAD,ACLS,ACMR,ACRS,ADBE,AEIS,AERI,AGEN,AGIO,AGNC,AIMC,AKAM,AKBA,ALGN,ALGT,ALKS,ALNY,ALRM,ALTR,ALXN,AMAT,AMBA,AMGN,AMKR,AMRS,AMZN,ANSS,APLS,APOP,APPF,APPN,ARCC,ARCT,ARGX,ARNA,ASML,...,UCTT,UFPI,ULTA,UPLD,URBN,UTHR,VCYT,VERI,VG,VIRT,VISL,VIVO,VNET,VRNS,VRNT,VRSK,VRSN,VRTX,VSAT,WB,WDFC,WERN,WING,WIX,WSC,WYNN,XLNX,XLRN,XNCR,XOMA,XONE,XRAY,YNDX,YY,ZBRA,ZG,ZGNX,ZKIN,ZLAB,ZNGA
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-02-03,10000,10000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2020-02-04,9993.05,0.190125,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2020-02-05,10782,0.190125,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2020-02-06,11031,0.229616,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2020-02-07,11568.7,1.78897,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-24,52909.8,0.481866,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2021-03-25,44984.3,1.09461,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2021-03-26,52141.6,3.45388,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
index = yf.Ticker('^IXIC')
index_df = index.history(start='2019-01-01')
index_df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2019-01-02,6506.910156,6693.709961,6506.879883,6665.939941,2261800000,0,0
2019-01-03,6584.770020,6600.209961,6457.129883,6463.500000,2607290000,0,0
2019-01-04,6567.140137,6760.689941,6554.240234,6738.859863,2579550000,0,0
2019-01-07,6757.529785,6855.600098,6741.399902,6823.470215,2507550000,0,0
2019-01-08,6893.439941,6909.580078,6795.859863,6897.000000,2380290000,0,0
...,...,...,...,...,...,...,...
2021-03-30,13008.799805,13075.750000,12922.570312,13045.389648,4685320000,0,0
2021-03-31,13122.570312,13325.540039,13118.379883,13246.870117,4980670000,0,0
2021-04-01,13414.320312,13487.080078,13404.179688,13480.110352,4553470000,0,0


In [11]:
results_df = fund_df.merge(index_df, how='left', left_index=True, right_index=True)
results_df.drop(results_df.columns.difference(['Fund Value','Close']), 1, inplace=True)
results_df.rename(columns={'Close': 'NASDAQ Composite'}, inplace=True)
nas_open = results_df['NASDAQ Composite'][0]
results_df['NASDAQ Composite'] = results_df['NASDAQ Composite']*10000 / nas_open
results_df.reset_index(inplace=True)
print(results_df)
results_df = results_df.melt('Date', var_name='Company', value_name='Price')

# Plots Results
line_plot = alt.Chart(results_df).mark_line().encode(
    x = 'Date',
    y = 'Price',
    color = 'Company'
).properties(
    title='Full Market Linear Regression Simulation Feb 2020-April 2021'
)
line_plot

          Date Fund Value  NASDAQ Composite
0   2020-02-03      10000      10000.000000
1   2020-02-04    9993.05      10209.814445
2   2020-02-05      10782      10253.714158
3   2020-02-06      11031      10322.157987
4   2020-02-07    11568.7      10266.471159
..         ...        ...               ...
287 2021-03-24    52909.8      13977.493802
288 2021-03-25    44984.3      13994.521040
289 2021-03-26    52141.6      14168.190648
290 2021-03-29    51393.4      14082.914401
291 2021-03-30    58802.8      14067.536285

[292 rows x 3 columns]


alt.Chart(...)

In [12]:
#fund_df.to_csv('simulation_results.csv')